In [13]:
import pandas as pd
from zipfile import ZipFile

with ZipFile("./Data Collection and Preprocessing/Processed data/past one year/contem_2023.csv.zip","r") as zip_ref:
    with zip_ref.open("contem_2023.csv") as file:
        contem_2023 = pd.read_csv(file)

In [14]:
contem_2023.head()

,date,rp_entity_id,comnam,ret,headline
0,2023-01-03,B4C673,YUM BRANDS INC,-0.008745,Business Disruptions Wane As Some Industries S...
1,2023-01-03,B4C673,YUM BRANDS INC,-0.008745,Press Release: KFC(R) Offers A Massive Deal on...
2,2023-01-03,B4C673,YUM BRANDS INC,-0.008745,Business News: Pizza Chains Race to Hire Driv...
3,2023-01-03,B4C673,YUM BRANDS INC,-0.008745,KFC(R) Offers A Massive Deal on a Comfort Food...
4,2023-01-03,B4C673,YUM BRANDS INC,-0.008745,Business Disruptions Wane as Some Industries S...


In [40]:
from gensim.models.ldamulticore import LdaMulticore
from gensim.corpora.dictionary import Dictionary
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm

headlines = contem_2023.headline.to_list()
tokenizer = CountVectorizer().build_tokenizer()
texts = [tokenizer(headline.lower()) for headline in tqdm(headlines)]
dictionary = Dictionary(texts)
corpous = [dictionary.doc2bow(text) for text in texts]

ldamodel = LdaMulticore(corpous,num_topics=62,id2word=dictionary,workers=100,passes = 5)


100%|██████████████████████████████| 1749770/1749770 [00:17<00:00, 99694.64it/s]


In [45]:
import numpy as np

topic_dist = np.zeros((contem_2023.shape[0],62))
for doc_id,bow in enumerate(corpous):
    for topic_id, prob in ldamodel.get_document_topics(bow,minimum_probability = 0):
        topic_dist[doc_id,topic_id] = prob

contem_2023_topic_dist = pd.concat([contem_2023.drop(columns = ["rp_entity_id","headline"]),\
                                    pd.DataFrame(topic_dist)], axis =1)
grouped_sum = contem_2023_topic_dist.groupby(["date","comnam","ret"]).sum()

X = np.array(grouped_sum)
ret = [ind[2] for ind in list(grouped_sum.index)]
Y = np.array(ret).reshape(-1,1)

from sklearn.linear_model import LinearRegression
lda_model = LinearRegression(fit_intercept=True)
lda_model.fit(X,Y)
lda_model.score(X,Y)

0.009739091532597888

In [48]:
import os

os.makedirs("/user/wx2309/Topic-modeling-store",exist_ok = True)
ldamodel.save("/user/wx2309/Topic-modeling-store/LDA_sp500_2023_contem_model")